<a href="https://colab.research.google.com/github/ToumaTanaka/Data_Science/blob/main/Tabel_Data/Machine_Learning/Supervised_Learning/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGBM
マイクロソフトが開発したGBDT(勾配ブースティング決定木)のモデル。  
決定木ベースなのでカテゴリー変数にLabel encodingを行っても正しく機能する。  

### LightGBMのパラメータについて

### LightGBMのハイパーパラメータについて


In [ ]:
import lightgbm as lgb

In [ ]:
#訓練データと評価用データを用意する

#カテゴリー変数を設定(カテゴリカル変数がある場合はそのリストを作る)
categorical_features = ['A','B','C']



#LightGBM用の訓練データセットを作成
lgb_train = lgb.Dataset(x_train,y_train,categorical_feature=categorical_features,free_raw_data=False)

#LightGBM用の評価データセットを作成
lgb_eval = lgb.Dataset(x_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features,free_raw_data=False)


#ハイパーパラメータを設定
params = {
    'boosting_type': 'gbdt',

    #'objective': 'binary', 二値分類
    'objective': 'regression', #回帰
    
    #'metric':'auc'　AUCの最大化を目指す
    'metric': {'l2', 'l1'}, 

    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'vervose': 0
    
}

#LightGBMの学習
model = lgb.train(params,lgb_train,
                 valid_sets=[lgb_train,lgb_eval],
                 verbose_eval=10,
                 num_boost_round=1000,
                 early_stopping_rounds= 20)
  

#LightGBMの予測
y_pred = model.predict(x_valid,num_interation=model.best_iteration)

クロスバリデーションver

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score


#カテゴリー変数を設定(カテゴリカル変数がある場合はそのリストを作る)
categorical_features = ['A','B','C']


#モデルを保存するリスト
models = []
#精度のスコアを保存するリスト
r2_scores = []

#KFoldクラスをインスタンス化
K_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)


#クロスバリデーションをする(しなくてもより)
for fold, (train_index, valid_index) in enumerate(K_fold.split(x,y)):

  x_train = x.iloc[train_index]
  x_valid = x.iloc[valid_index]
  y_train = y.iloc[train_index]
  y_valid = y.iloc[valid_index]

　　　#LightGBM用の訓練データセットを作成
  lgb_train = lgb.Dataset(x_train,y_train,categorical_feature=categorical_features,free_raw_data=False)

　　　#LightGBM用の評価データセットを作成
  lgb_eval = lgb.Dataset(x_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features,free_raw_data=False)


　　　#ハイパーパラメータを設定
  params = {
    'boosting_type': 'gbdt',

    #'objective': 'binary', 二値分類
    'objective': 'regression', #回帰
    
    #'metric':'auc'　AUCの最大化を目指す
    'metric': {'l2', 'l1'}, 

    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'vervose': 0
    
  }

　　　#LightGBMの学習
  model = lgb.train(params,lgb_train,
                 valid_sets=[lgb_train,lgb_eval],
                 verbose_eval=10,
                 num_boost_round=1000,
                 early_stopping_rounds= 20)
  

  #LightGBMの予測
  y_pred = model.predict(x_valid,num_interation=model.best_iteration)


　　　#r2_scoreで評価
  score = r2_score(y_valid,y_pred)
  
  r2_scores.append(score)
  models.append(model)